# II/ Seq2seq with attention

In [1]:
# TODO: https://towardsdatascience.com/transformers-141e32e69591

## A) Attention in general

## B) Bahdanau attention

In [2]:
# TODO: https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html

## C) CNN + attention

In [1]:
# TODO: https://github.com/bentrevett/pytorch-seq2seq/blob/master/5%20-%20Convolutional%20Sequence%20to%20Sequence%20Learning.ipynb

## D) Towards an unified model of attention